In [1]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import os
from torchvision import datasets, transforms

In [2]:
num_classes = 3

In [3]:
model = torchvision.models.efficientnet_v2_s(weights=None)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

In [7]:
model = torchvision.models.convnext_tiny(weights=None)
model.classifier[2] = nn.Linear(model.classifier[2].in_features, num_classes)

In [3]:
model = torchvision.models.vit_b_16(weights=torchvision.models.ViT_B_16_Weights.DEFAULT)

for param in model.parameters():
    param.requires_grad = False

classifier = nn.Sequential(
    nn.Linear(model.heads[0].in_features, model.heads[0].in_features),
    nn.ReLU(True),
    nn.Linear(model.heads[0].in_features, num_classes)
)

model.heads = classifier

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /home/tho121/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100.0%


In [4]:
model = torchvision.models.swin_v2_t(weights=torchvision.models.Swin_V2_T_Weights.DEFAULT)

for param in model.parameters():
    param.requires_grad = False

classifier = nn.Sequential(
    nn.Linear(model.head.in_features, model.head.in_features),
    nn.ReLU(True),
    nn.Linear(model.head.in_features, num_classes)
)

model.head = classifier

In [4]:
train_transforms = transforms.Compose([
        #transforms.RandomSizedCrop(224),
        transforms.RandomCrop(224, 64),
        transforms.RandomRotation((-90,90)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

test_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

In [5]:
train_dir = 'dataset/train'
train_image_datasets = datasets.ImageFolder(train_dir, train_transforms)

val_dir = 'dataset/val'
val_image_datasets = datasets.ImageFolder(val_dir, test_transforms)

test_dir = 'dataset/test'
test_image_datasets = datasets.ImageFolder(test_dir, test_transforms)

In [6]:
#efficientnet_v2_s = 32
#vit_b_16 = 24

batch_size = 24

In [7]:
train_loader = torch.utils.data.DataLoader(train_image_datasets,
                                             batch_size=batch_size, shuffle=True,
                                             num_workers=2)

val_loader = torch.utils.data.DataLoader(val_image_datasets,
                                             batch_size=batch_size, shuffle=True,
                                             num_workers=2)

test_loader = torch.utils.data.DataLoader(test_image_datasets,
                                             batch_size=batch_size, shuffle=True,
                                             num_workers=2)

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [10]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter

In [11]:
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=0.0005) #effnet, vit = 0.0005
criteron = nn.CrossEntropyLoss()
lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)

writer = SummaryWriter('output')

# let's train it for 10 epochs
num_epochs = 200
best_acc = 0
best_epoch = 0

dataloaders = {}
dataloaders['train'] = train_loader
dataloaders['val'] = val_loader

dataset_sizes = {}
dataset_sizes['train'] = len(train_image_datasets)
dataset_sizes['val'] = len(val_image_datasets)

for epoch in range(num_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            if (epoch+1) % 5 > 0:  #only every 5 epochs
                continue

            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criteron(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step(loss.item)

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        if phase == 'train':
            writer.add_scalar('training loss', epoch_loss, epoch)
            writer.add_scalar('training acc', epoch_acc, epoch)
            lr_scheduler.step(epoch_loss)
        elif phase == 'val':
            writer.add_scalar('val loss', epoch_loss, epoch)
            writer.add_scalar('val acc', epoch_acc, epoch)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        writer.add_scalar('learning rate', optimizer.param_groups[0]['lr'], epoch)

        # save best model
        if epoch > 10 and phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            print("Saving at epoch: %.1f" % (epoch))
            best_epoch = epoch
            torch.save(model.state_dict(), 'best_checkpoint.pt')

print("finished")
writer.close()
torch.save(model.state_dict(), 'final_checkpoint.pt')
print("Best epoch: %.1f" % (best_epoch))

train Loss: 0.3404 Acc: 0.8596
train Loss: 0.1959 Acc: 0.9121
train Loss: 0.1782 Acc: 0.9242
train Loss: 0.1626 Acc: 0.9317
train Loss: 0.1467 Acc: 0.9388
val Loss: 0.0819 Acc: 0.9633
train Loss: 0.1443 Acc: 0.9438
train Loss: 0.1518 Acc: 0.9304
train Loss: 0.1341 Acc: 0.9463
train Loss: 0.1207 Acc: 0.9483
train Loss: 0.1278 Acc: 0.9504
val Loss: 0.0735 Acc: 0.9700
train Loss: 0.1323 Acc: 0.9446
train Loss: 0.1375 Acc: 0.9471
train Loss: 0.1074 Acc: 0.9592
train Loss: 0.1119 Acc: 0.9492
train Loss: 0.1192 Acc: 0.9450
val Loss: 0.0692 Acc: 0.9700
Saving at epoch: 14.0
train Loss: 0.1135 Acc: 0.9567
train Loss: 0.1174 Acc: 0.9521
train Loss: 0.1136 Acc: 0.9563
train Loss: 0.1106 Acc: 0.9550
train Loss: 0.1147 Acc: 0.9513
val Loss: 0.0505 Acc: 0.9833
Saving at epoch: 19.0
train Loss: 0.1090 Acc: 0.9508
train Loss: 0.1095 Acc: 0.9554
train Loss: 0.1166 Acc: 0.9500
train Loss: 0.1093 Acc: 0.9550
train Loss: 0.0996 Acc: 0.9596
val Loss: 0.0841 Acc: 0.9767
train Loss: 0.1063 Acc: 0.9588
train